In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import *
from sklearn.model_selection import *
from sklearn.decomposition import PCA
from datetime import datetime
from imblearn.over_sampling import SMOTENC,SMOTE
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import EditedNearestNeighbours 
from collections import Counter
from sklearn.metrics import *
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score, auc
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
weather_full=pd.read_csv("../Data/weather_data_2.csv")
weather_full

,timestamp,region,past_temperature,past_humidity,past_rainfall,past_wind_x,past_wind_y,delta_temperature,delta_humidity,delta_wind_x,delta_wind_y,rainfall
0,2017-01-01 03:00:00,central,26.850000,90.300000,0.0,-6.0,-4.0,-0.100000,-1.421085e-14,0.0,0.0,0.0
1,2017-01-01 03:00:00,east,26.125000,87.433333,0.0,-1.0,1.0,-0.150000,-5.333333e-01,0.0,1.0,0.0
2,2017-01-01 03:00:00,north,26.000000,87.000000,0.0,-1.0,2.0,-0.100000,4.000000e-01,0.0,0.0,0.0
3,2017-01-01 03:00:00,north-east,26.000000,89.250000,0.0,-1.0,1.0,0.033333,-1.500000e-01,-2.0,-1.0,0.0
4,2017-01-01 03:00:00,west,26.100000,87.066667,0.0,-4.0,0.0,-0.140000,8.666667e-01,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
197635,2021-12-31 23:00:00,central,23.666667,95.400000,1.0,-1.0,-1.0,-0.766667,-2.150000e+00,-3.0,1.0,1.0
197636,2021-12-31 23:00:00,east,24.800000,93.000000,1.0,-3.0,0.0,0.100000,-3.000000e-01,-2.0,-2.0,0.0
197637,2021-12-31 23:00:00,north,24.500000,96.200000,1.0,0.0,-1.0,0.200000,-6.000000e-01,-2.0,-1.0,0.0
197638,2021-12-31 23:00:00,north-east,24.300000,92.650000,0.0,0.0,-1.0,0.350000,-1.000000e+00,0.0,-3.0,0.0


Knowing more about the data, descriptive stats

In [ ]:
weather_full.describe()

,past_temperature,past_humidity,past_rainfall,past_wind_x,past_wind_y,delta_temperature,delta_humidity,delta_wind_x,delta_wind_y,rainfall
count,197640.000000,197640.000000,197640.000000,197640.000000,197640.000000,197640.000000,197640.000000,197640.000000,197640.000000,197640.000000
mean,27.981512,79.881846,0.083010,0.805227,-0.687715,-0.005856,0.021792,-0.000562,0.000769,0.083354
std,2.100826,10.931622,0.275897,3.073347,3.637827,0.793041,4.340665,2.350381,2.839940,0.276417
min,21.400000,32.966667,0.000000,-27.000000,-24.000000,-8.633333,-28.650000,-30.000000,-30.000000,0.000000
25%,26.400000,72.566667,0.000000,-1.000000,-3.000000,-0.300000,-2.000000,-1.000000,-1.000000,0.000000
50%,27.766667,82.000000,0.000000,1.000000,-1.000000,-0.066667,0.250000,0.000000,0.000000,0.000000
75%,29.450000,88.266667,0.000000,3.000000,2.000000,0.333333,1.933333,1.000000,1.000000,0.000000
max,35.800000,101.000000,1.000000,25.000000,19.000000,5.700000,39.566667,32.000000,29.000000,1.000000


In [ ]:
weather_full.shape

(197640, 12)

checking distribution of classes

In [ ]:
weather_full['rainfall'].value_counts()

0.0    181166
1.0     16474
Name: rainfall, dtype: int64

Creating time variables

In [ ]:
weather_full.timestamp=pd.to_datetime(weather_full.timestamp,infer_datetime_format=True)
weather_full["year"]=weather_full.timestamp.apply(lambda x: x.year)
weather_full["quarter"]=weather_full.timestamp.apply(lambda x: x.quarter)
weather_full["month"]=weather_full.timestamp.apply(lambda x: x.month)
weather_full["day"]=weather_full.timestamp.apply(lambda x: x.day)
weather_full["hour"]=weather_full.timestamp.apply(lambda x: x.hour)
weather_full

,timestamp,region,past_temperature,past_humidity,past_rainfall,past_wind_x,past_wind_y,delta_temperature,delta_humidity,delta_wind_x,delta_wind_y,rainfall,year,quarter,month,day,hour
0,2017-01-01 03:00:00,central,26.850000,90.300000,0.0,-6.0,-4.0,-0.100000,-1.421085e-14,0.0,0.0,0.0,2017,1,1,1,3
1,2017-01-01 03:00:00,east,26.125000,87.433333,0.0,-1.0,1.0,-0.150000,-5.333333e-01,0.0,1.0,0.0,2017,1,1,1,3
2,2017-01-01 03:00:00,north,26.000000,87.000000,0.0,-1.0,2.0,-0.100000,4.000000e-01,0.0,0.0,0.0,2017,1,1,1,3
3,2017-01-01 03:00:00,north-east,26.000000,89.250000,0.0,-1.0,1.0,0.033333,-1.500000e-01,-2.0,-1.0,0.0,2017,1,1,1,3
4,2017-01-01 03:00:00,west,26.100000,87.066667,0.0,-4.0,0.0,-0.140000,8.666667e-01,0.0,0.0,0.0,2017,1,1,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197635,2021-12-31 23:00:00,central,23.666667,95.400000,1.0,-1.0,-1.0,-0.766667,-2.150000e+00,-3.0,1.0,1.0,2021,4,12,31,23
197636,2021-12-31 23:00:00,east,24.800000,93.000000,1.0,-3.0,0.0,0.100000,-3.000000e-01,-2.0,-2.0,0.0,2021,4,12,31,23
197637,2021-12-31 23:00:00,north,24.500000,96.200000,1.0,0.0,-1.0,0.200000,-6.000000e-01,-2.0,-1.0,0.0,2021,4,12,31,23
197638,2021-12-31 23:00:00,north-east,24.300000,92.650000,0.0,0.0,-1.0,0.350000,-1.000000e+00,0.0,-3.0,0.0,2021,4,12,31,23


In [ ]:
weather_full=pd.concat([weather_full.iloc[:,12:],weather_full.iloc[:,:11],weather_full.iloc[:,11:12]],axis=1)
weather_full

,year,quarter,month,day,hour,timestamp,region,past_temperature,past_humidity,past_rainfall,past_wind_x,past_wind_y,delta_temperature,delta_humidity,delta_wind_x,delta_wind_y,rainfall
0,2017,1,1,1,3,2017-01-01 03:00:00,central,26.850000,90.300000,0.0,-6.0,-4.0,-0.100000,-1.421085e-14,0.0,0.0,0.0
1,2017,1,1,1,3,2017-01-01 03:00:00,east,26.125000,87.433333,0.0,-1.0,1.0,-0.150000,-5.333333e-01,0.0,1.0,0.0
2,2017,1,1,1,3,2017-01-01 03:00:00,north,26.000000,87.000000,0.0,-1.0,2.0,-0.100000,4.000000e-01,0.0,0.0,0.0
3,2017,1,1,1,3,2017-01-01 03:00:00,north-east,26.000000,89.250000,0.0,-1.0,1.0,0.033333,-1.500000e-01,-2.0,-1.0,0.0
4,2017,1,1,1,3,2017-01-01 03:00:00,west,26.100000,87.066667,0.0,-4.0,0.0,-0.140000,8.666667e-01,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197635,2021,4,12,31,23,2021-12-31 23:00:00,central,23.666667,95.400000,1.0,-1.0,-1.0,-0.766667,-2.150000e+00,-3.0,1.0,1.0
197636,2021,4,12,31,23,2021-12-31 23:00:00,east,24.800000,93.000000,1.0,-3.0,0.0,0.100000,-3.000000e-01,-2.0,-2.0,0.0
197637,2021,4,12,31,23,2021-12-31 23:00:00,north,24.500000,96.200000,1.0,0.0,-1.0,0.200000,-6.000000e-01,-2.0,-1.0,0.0
197638,2021,4,12,31,23,2021-12-31 23:00:00,north-east,24.300000,92.650000,0.0,0.0,-1.0,0.350000,-1.000000e+00,0.0,-3.0,0.0


In [ ]:
runtimes=list(pd.date_range('2017-01-01 00:00:00',
                            '2021-12-31 23:59:59',
                            freq='60T'))
training_runtimes=runtimes[:int(0.8*len(runtimes))]
X_train = weather_full[weather_full["timestamp"].isin(training_runtimes)].iloc[:,:-1]
X_test = weather_full[~weather_full["timestamp"].isin(training_runtimes)].iloc[:,:-1]
y_train = weather_full[weather_full["timestamp"].isin(training_runtimes)][["rainfall"]]
y_test = weather_full[~weather_full["timestamp"].isin(training_runtimes)][["rainfall"]]
X_train=X_train.drop(columns=["timestamp"])
X_test=X_test.drop(columns=["timestamp"])
X_train

,year,quarter,month,day,hour,region,past_temperature,past_humidity,past_rainfall,past_wind_x,past_wind_y,delta_temperature,delta_humidity,delta_wind_x,delta_wind_y
0,2017,1,1,1,3,central,26.850000,90.300000,0.0,-6.0,-4.0,-0.100000,-1.421085e-14,0.0,0.0
1,2017,1,1,1,3,east,26.125000,87.433333,0.0,-1.0,1.0,-0.150000,-5.333333e-01,0.0,1.0
2,2017,1,1,1,3,north,26.000000,87.000000,0.0,-1.0,2.0,-0.100000,4.000000e-01,0.0,0.0
3,2017,1,1,1,3,north-east,26.000000,89.250000,0.0,-1.0,1.0,0.033333,-1.500000e-01,-2.0,-1.0
4,2017,1,1,1,3,west,26.100000,87.066667,0.0,-4.0,0.0,-0.140000,8.666667e-01,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156169,2020,4,12,31,18,central,25.960000,89.800000,1.0,0.0,-1.0,-0.080000,1.850000e+00,1.0,2.0
156170,2020,4,12,31,18,east,29.300000,66.200000,0.0,3.0,5.0,-0.200000,6.000000e-01,0.0,0.0
156171,2020,4,12,31,18,north,25.650000,83.300000,1.0,-3.0,3.0,-2.700000,1.085000e+01,-10.0,7.0
156172,2020,4,12,31,18,north-east,28.333333,74.800000,0.0,4.0,-1.0,-0.833333,4.433333e+00,0.0,-4.0


smote resampling - to handle the class imbalances

In [ ]:
print(Counter(y_train.rainfall))
smote_nc = SMOTENC(categorical_features=[5], random_state=0)
X_resampled, y_resampled = smote_nc.fit_resample(X_train, y_train.rainfall)
Counter(y_resampled)

Counter({0.0: 143549, 1.0: 12625})


Counter({0.0: 143549, 1.0: 143549})

In [ ]:
X_resampled

,year,quarter,month,day,hour,region,past_temperature,past_humidity,past_rainfall,past_wind_x,past_wind_y,delta_temperature,delta_humidity,delta_wind_x,delta_wind_y
0,2017,1,1,1,3,central,26.850000,90.300000,0.000000,-6.000000,-4.000000,-0.100000,-1.421085e-14,0.000000,0.000000
1,2017,1,1,1,3,east,26.125000,87.433333,0.000000,-1.000000,1.000000,-0.150000,-5.333333e-01,0.000000,1.000000
2,2017,1,1,1,3,north,26.000000,87.000000,0.000000,-1.000000,2.000000,-0.100000,4.000000e-01,0.000000,0.000000
3,2017,1,1,1,3,north-east,26.000000,89.250000,0.000000,-1.000000,1.000000,0.033333,-1.500000e-01,-2.000000,-1.000000
4,2017,1,1,1,3,west,26.100000,87.066667,0.000000,-4.000000,0.000000,-0.140000,8.666667e-01,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287093,2018,2,6,6,10,north-east,26.018087,93.458274,0.713579,2.286421,-2.286421,-1.694997,8.821806e+00,0.000000,-3.286421
287094,2017,2,6,18,9,west,28.372432,82.265984,0.000000,-5.108495,-3.108495,-0.204401,2.121119e+00,-3.847491,-1.261004
287095,2017,1,1,2,14,central,26.551796,81.207136,1.000000,6.657089,-0.328545,-0.309497,-1.882089e+00,7.342911,4.000000
287096,2019,2,6,3,19,east,28.960638,81.380277,1.000000,-4.852709,-1.803612,-2.009128,9.654007e+00,-0.901806,0.049097


In [ ]:
temp_df=pd.concat([X_resampled,X_test],axis=0)
temp_df=pd.get_dummies(temp_df, columns=["region"], prefix=["region"])
scaler=StandardScaler()
temp_df.iloc[:,5:14]=scaler.fit_transform(temp_df.iloc[:,5:14])
X_resampled = temp_df.iloc[:len(X_resampled),:]
X_resampled

,year,quarter,month,day,hour,past_temperature,past_humidity,past_rainfall,past_wind_x,past_wind_y,delta_temperature,delta_humidity,delta_wind_x,delta_wind_y,region_central,region_east,region_north,region_north-east,region_west
0,2017,1,1,1,3,-0.368775,0.785725,-0.619154,-1.927747,-0.908897,0.076037,-0.151417,0.016280,0.017967,1,0,0,0,0
1,2017,1,1,1,3,-0.693481,0.525594,-0.619154,-0.405321,0.536957,0.025337,-0.258075,0.016280,0.354667,0,1,0,0,0
2,2017,1,1,1,3,-0.749465,0.486272,-0.619154,-0.405321,0.826128,0.076037,-0.071423,0.016280,0.017967,0,0,1,0,0
3,2017,1,1,1,3,-0.749465,0.690445,-0.619154,-0.405321,0.536957,0.211238,-0.181414,-0.730024,-0.318733,0,0,0,1,0
4,2017,1,1,1,3,-0.704678,0.492322,-0.619154,-1.318777,0.247786,0.035477,0.021903,0.016280,0.017967,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287093,2018,2,6,6,10,-0.741365,1.072317,1.108104,0.595346,-0.413380,-1.541297,1.612804,0.016280,-1.088571,0,0,0,1,0
287094,2017,2,6,18,9,0.313078,0.056692,-0.619154,-1.656297,-0.651100,-0.029826,0.272773,-1.419418,-0.406613,0,0,0,0,1
287095,2017,1,1,2,14,-0.502332,-0.039392,1.801403,1.926150,0.152781,-0.136394,-0.527804,2.756299,1.364767,1,0,0,0,0
287096,2019,2,6,3,19,0.576518,-0.023680,1.801403,-1.578414,-0.273766,-1.859828,1.779230,-0.320231,0.034498,0,1,0,0,0


In [ ]:
X_test = temp_df.iloc[len(X_resampled):,:]
del temp_df
X_test

,year,quarter,month,day,hour,past_temperature,past_humidity,past_rainfall,past_wind_x,past_wind_y,delta_temperature,delta_humidity,delta_wind_x,delta_wind_y,region_central,region_east,region_north,region_north-east,region_west
156174,2020,4,12,31,19,-1.448144,1.300693,1.801403,0.812620,-0.041385,-1.363849,1.083485,1.135735,0.017967,1,0,0,0,0
156175,2020,4,12,31,19,0.370212,-0.838581,-0.619154,1.726076,1.982812,-0.633766,1.088484,1.135735,0.354667,0,1,0,0,0
156176,2020,4,12,31,19,-1.040581,0.658684,1.801403,0.812620,0.247786,-0.126764,0.968494,2.255190,-0.992133,0,0,1,0,0
156177,2020,4,12,31,19,-0.674820,0.519545,1.801403,0.203650,-0.619726,-2.019573,2.361716,-1.103175,-0.655433,0,0,0,1,0
156178,2020,4,12,31,19,-1.436201,1.157773,1.801403,1.117105,0.826128,-0.363365,-0.058091,1.135735,2.711566,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197635,2021,4,12,31,23,-1.794498,1.248516,1.801403,-0.405321,-0.041385,-0.599966,-0.581382,-1.103175,0.354667,1,0,0,0,0
197636,2021,4,12,31,23,-1.286911,1.030732,1.801403,-1.014291,0.247786,0.278838,-0.211412,-0.730024,-0.655433,0,1,0,0,0
197637,2021,4,12,31,23,-1.421272,1.321111,1.801403,-0.100836,-0.041385,0.380239,-0.271407,-0.730024,-0.318733,0,0,1,0,0
197638,2021,4,12,31,23,-1.510846,0.998972,-0.619154,-0.100836,-0.041385,0.532340,-0.351401,0.016280,-0.992133,0,0,0,1,0


In [ ]:
#Printing the shape of each resampled and test dataset
print(X_resampled.shape, X_test.shape, '\n', y_resampled.shape, y_test.shape)

(287098, 19) (41466, 19) 
 (287098,) (41466, 1)


In [ ]:
X_resampled.shape

(287098, 19)

In [ ]:
X_test.shape

(41466, 19)

In [ ]:
type(X_test)

pandas.core.frame.DataFrame

In [ ]:
y_resampled.shape

(287098,)

In [ ]:
type(y_resampled)

pandas.core.series.Series

In [ ]:
y_test.shape

(41466, 1)

NOTE: Training is now called "resampled" as train is the imbalanced (and undumified) data

In [ ]:
svm_model=LinearSVC(dual= False)

svm_model.fit(X_resampled, y_resampled)
svm_model_pred = svm_model.predict(X_test)

print('Confusion Matrix :', '\n', confusion_matrix(svm_model_pred, y_test))

# #Accuracy score
asr_svm_1 = accuracy_score(y_test,svm_model_pred)
print('Accuracy score for SVM : ', asr_svm_1)

# #F score
f1_svm_1 = f1_score(y_test,svm_model_pred)
print('F-score for SVM : ', f1_svm_1)

# #Precision
precision_svm_1 = precision_score(y_test, svm_model_pred) 
print('Precision for SVM : ', precision_svm_1)

# #Recall
recall_svm_1 = recall_score(y_test, svm_model_pred) 
print('Recall for SVM : ', recall_svm_1)

# #AUC 
fpr, tpr, threshold = roc_curve(y_test, svm_model_pred)
roc_auc = auc(fpr, tpr)
print("area under curve (auc): ", roc_auc)


Confusion Matrix : 
 [[35277  1666]
 [ 2340  2183]]
Accuracy score for SVM :  0.9033907297544976
F-score for SVM :  0.5215002388915432
Precision for SVM :  0.4826442626575282
Recall for SVM :  0.567160301376981
area under curve (auc):  0.7524771919198486


In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {'C':[0.01, 0.05, 0.1] }
grid_search= GridSearchCV(estimator = svm_model,
                         param_grid = parameters,
                         scoring = 'f1',
                         cv = 10,
                         n_jobs = -1,
                         verbose=20)

grid_search= grid_search.fit(X_resampled, y_resampled)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_

print("Best Accuracy : {:.2f} %".format(best_accuracy*100))
print("Best Paramters : ", best_parameters)

Fitting 10 folds for each of 3 candidates, totalling 30 fits
Best Accuracy : 72.68 %
Best Paramters :  {'C': 0.01}


In [ ]:
#New linear SVM with C = 0.01
svm_model_tuned=LinearSVC(dual= False, C=0.01)

svm_model_tuned.fit(X_resampled, y_resampled)
svm_model_pred = svm_model_tuned.predict(X_test)

# #confusion matrix
print('Confusion Matrix :', '\n', confusion_matrix(svm_model_pred, y_test))

# #Accuracy score
asr_svm_1 = accuracy_score(y_test,svm_model_pred)
print('Accuracy score for SVM : ', asr_svm_1)

# #F score
f1_svm_1 = f1_score(y_test,svm_model_pred)
print('F-score for SVM : ', f1_svm_1)

# #Precision
precision_svm_1 = precision_score(y_test, svm_model_pred) 
print('Precision for SVM : ', precision_svm_1)

# #Recall
recall_svm_1 = recall_score(y_test, svm_model_pred) 
print('Recall for SVM : ', recall_svm_1)

# #AUC 
auc_svm_1 = roc_auc_score(y_test, svm_model_pred)

print("area under curve (auc): ",  auc_svm_1)

Confusion Matrix : 
 [[35271  1666]
 [ 2346  2183]]
Accuracy score for SVM :  0.9032460328944195
F-score for SVM :  0.5211267605633803
Precision for SVM :  0.4820048575844557
Recall for SVM :  0.567160301376981
area under curve (auc):  0.7523974407435188


In [ ]:
parameters = {'max_iter':[100, 500, 1000] }
grid_search= GridSearchCV(estimator = svm_model_tuned,
                         param_grid = parameters,
                         scoring = 'f1',
                         cv = 10,
                         n_jobs = -1,
                         verbose=20)

grid_search= grid_search.fit(X_resampled, y_resampled)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_

print("Best Accuracy : {:.2f} %".format(best_accuracy*100))
print("Best Paramters : ", best_parameters)

Fitting 10 folds for each of 3 candidates, totalling 30 fits
Best Accuracy : 72.68 %
Best Paramters :  {'max_iter': 100}


In [ ]:
svm_model_tuned_1=LinearSVC(dual= False, C=0.01, max_iter= 100)

svm_model_tuned_1.fit(X_resampled, y_resampled)
svm_model_pred = svm_model_tuned_1.predict(X_test)

# #confusion matrix
print('Confusion Matrix :', '\n', confusion_matrix(svm_model_pred, y_test))

# #Accuracy score
asr_svm_1 = accuracy_score(y_test,svm_model_pred)
print('Accuracy score for SVM : ', asr_svm_1)

# #F score
f1_svm_1 = f1_score(y_test,svm_model_pred)
print('F-score for SVM : ', f1_svm_1)

# #Precision
precision_svm_1 = precision_score(y_test, svm_model_pred) 
print('Precision for SVM : ', precision_svm_1)

# #Recall
recall_svm_1 = recall_score(y_test, svm_model_pred) 
print('Recall for SVM : ', recall_svm_1)

# #AUC 
auc_svm_1 = roc_auc_score(y_test, svm_model_pred)

print("area under curve (auc): ", auc_svm_1)

Confusion Matrix : 
 [[35271  1666]
 [ 2346  2183]]
Accuracy score for SVM :  0.9032460328944195
F-score for SVM :  0.5211267605633803
Precision for SVM :  0.4820048575844557
Recall for SVM :  0.567160301376981
area under curve (auc):  0.7523974407435188


In [ ]:
parameters = {'tol':[1e-4, 1e-5, 1e-6] }
grid_search= GridSearchCV(estimator = svm_model_tuned_1,
                         param_grid = parameters,
                         scoring = 'f1',
                         cv = 10,
                         n_jobs = -1,
                         verbose=20)

grid_search= grid_search.fit(X_resampled, y_resampled)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_

print("Best Accuracy : {:.2f} %".format(best_accuracy*100))
print("Best Paramters : ", best_parameters)

Fitting 10 folds for each of 3 candidates, totalling 30 fits
Best Accuracy : 72.68 %
Best Paramters :  {'tol': 1e-05}


In [ ]:
svm_model_tuned_2=LinearSVC(dual= False, C=0.01, max_iter= 100, tol = 1e-5)

svm_model_tuned_2.fit(X_resampled, y_resampled)
svm_model_pred = svm_model_tuned_2.predict(X_test)

# #confusion matrix
print('Confusion Matrix :', '\n', confusion_matrix(svm_model_pred, y_test))

# #Accuracy score
asr_svm_1 = accuracy_score(y_test,svm_model_pred)
print('Accuracy score for SVM : ', asr_svm_1)

# #F score
f1_svm_1 = f1_score(y_test,svm_model_pred)
print('F-score for SVM : ', f1_svm_1)

# #Precision
precision_svm_1 = precision_score(y_test, svm_model_pred) 
print('Precision for SVM : ', precision_svm_1)

# #Recall
recall_svm_1 = recall_score(y_test, svm_model_pred) 
print('Recall for SVM : ', recall_svm_1)

# #AUC 
fpr, tpr, threshold = roc_curve(y_test, svm_model_pred)
roc_auc = auc(fpr, tpr)
print("area under curve (auc): ", roc_auc)

Confusion Matrix : 
 [[35271  1666]
 [ 2346  2183]]
Accuracy score for SVM :  0.9032460328944195
F-score for SVM :  0.5211267605633803
Precision for SVM :  0.4820048575844557
Recall for SVM :  0.567160301376981
area under curve (auc):  0.7523974407435188


In [ ]:
sorted(svm_model.get_params().keys())

['C',
 'class_weight',
 'dual',
 'fit_intercept',
 'intercept_scaling',
 'loss',
 'max_iter',
 'multi_class',
 'penalty',
 'random_state',
 'tol',
 'verbose']

# AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

abc = AdaBoostClassifier(base_estimator= svm_model, learning_rate= 0.01, n_estimators= 2, algorithm='SAMME')
abc.fit(X_resampled, y_resampled)
y_pred = abc.predict(X_test)

cm_abc = confusion_matrix(y_test, y_pred)
asr_abc = accuracy_score(y_test, y_pred)
f1_abc = f1_score(y_test, y_pred)
precision_abc = precision_score(y_test, y_pred) 
recall_abc = recall_score(y_test, y_pred)     
       
print('Confusion Matrix : ', '\n', cm_abc)
print('Accuracy score : ', asr_abc)
print('F1 score : ', f1_abc)
print('Precision score : ', precision_abc)
print('Recall score : ', recall_abc)

y_preds= abc.predict_proba(X_test)
fpr, tpr , threshold = roc_curve(y_test, y_preds[:,1,None], pos_label=1)
print('Area under curve : ', auc(fpr,tpr))

Confusion Matrix :  
 [[35683  1934]
 [ 1715  2134]]
Accuracy score :  0.9120001929291468
F1 score :  0.5390930908172287
Precision score :  0.5245821042281219
Recall score :  0.5544297220057157
Area under curve :  0.7515072519940263


In [ ]:
#Tunning adaboost model

parameters = {'learning_rate':[0.01, 0.05, 0.1] }
grid_search= GridSearchCV(estimator = abc,
                         param_grid = parameters,
                         scoring = 'f1',
                         cv = 10,
                         n_jobs = -1,
                         verbose=20)

grid_search= grid_search.fit(X_resampled, y_resampled)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_

print("Best Accuracy : {:.2f} %".format(best_accuracy*100))
print("Best Paramters : ", best_parameters)

Fitting 10 folds for each of 3 candidates, totalling 30 fits
Best Accuracy : 72.11 %
Best Paramters :  {'learning_rate': 0.01}


In [ ]:
parameters = {'n_estimators':[1,2,3,4] }
grid_search= GridSearchCV(estimator = abc,
                         param_grid = parameters,
                         scoring = 'f1',
                         cv = 10,
                         n_jobs = -1,
                         verbose=20)

grid_search= grid_search.fit(X_resampled, y_resampled)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_

print("Best Accuracy : {:.2f} %".format(best_accuracy*100))
print("Best Paramters : ", best_parameters)

Fitting 10 folds for each of 4 candidates, totalling 40 fits
Best Accuracy : 72.16 %
Best Paramters :  {'n_estimators': 3}


In [ ]:
# (FINAL) TUNED ADABOOST MODEL 

abc2 = AdaBoostClassifier(base_estimator= svm_model, learning_rate= 0.01, n_estimators= 3, algorithm='SAMME')
abc2.fit(X_resampled, y_resampled)
y_pred = abc2.predict(X_test)

cm_abc = confusion_matrix(y_test, y_pred)
asr_abc = accuracy_score(y_test, y_pred)
f1_abc = f1_score(y_test, y_pred)
precision_abc = precision_score(y_test, y_pred) 
recall_abc = recall_score(y_test, y_pred)     
       
print('Confusion Matrix : ', '\n', cm_abc)
print('Accuracy score : ', asr_abc)
print('F1 score : ', f1_abc)
print('Precision score : ', precision_abc)
print('Recall score : ', recall_abc)

y_preds= abc.predict_proba(X_test)
fpr, tpr , threshold = roc_curve(y_test, y_preds[:,1,None], pos_label=1)
print('Area under curve : ', auc(fpr,tpr))

Confusion Matrix :  
 [[35662  1955]
 [ 1714  2135]]
Accuracy score :  0.9115178700622196
F1 score :  0.5378511147499686
Precision score :  0.5220048899755502
Recall score :  0.5546895297479865
Area under curve :  0.7515072519940263


### Creating the dataframe for PCA variables

In [ ]:
temp_df=pd.concat([X_resampled,X_test],axis=0)
pca=PCA(n_components=0.95)
pca_components=pca.fit_transform(temp_df)
pca_df = pd.DataFrame(pca_components,columns=["PC1","PC2","PC3","PC4","PC5"])
pca_df

,PC1,PC2,PC3,PC4,PC5
0,-14.880755,8.463824,5.761134,0.829468,-2.126599
1,-14.880415,8.460514,5.871510,0.559080,-1.282495
2,-14.880803,8.459678,5.884406,0.630299,-1.246453
3,-14.880274,8.481644,5.869281,0.646356,-1.466767
4,-14.880726,8.463801,5.827905,0.841523,-1.608660
...,...,...,...,...,...
328559,15.687172,-10.601566,-5.460768,1.769499,2.624000
328560,15.686689,-10.615775,-5.457656,1.271116,2.304801
328561,15.686498,-10.608580,-5.439477,1.245831,2.443421
328562,15.687299,-10.596882,-5.417433,-0.051533,2.211964


## (Linear) SVM PCA

In [ ]:
#initialise SVM Linear
clf_pca = LinearSVC(dual= False) 

#train model
clf_pca.fit(pca_df.iloc[:len(X_resampled),:], y_resampled)

LinearSVC(dual=False)

In [ ]:
#get predictions and compare results
predictions_pca = svm_model_tuned_2.predict(X_test)

In [ ]:
cm_pca = confusion_matrix(y_test, predictions_pca)
asr_pca = accuracy_score(y_test, predictions_pca)
f1_pca = f1_score(y_test, predictions_pca)
precision_pca = precision_score(y_test, predictions_pca) 
recall_pca = recall_score(y_test, predictions_pca)     
       
print('Confusion Matrix : ', '\n', cm_pca)
print('Accuracy score : ', asr_pca)
print('F1 score : ', f1_pca)
print('Precision score : ', precision_pca)
print('Recall score : ', recall_pca)

Confusion Matrix :  
 [[35271  2346]
 [ 1666  2183]]
Accuracy score :  0.9032460328944195
F1 score :  0.5211267605633803
Precision score :  0.4820048575844557
Recall score :  0.567160301376981


In [ ]:
clf_pca_pred =clf_pca.predict(pca_df.iloc[:len(X_test),:])
fpr, tpr, thresholds = roc_curve(y_test, clf_pca_pred, pos_label=1)
print('Area under curve : ', auc(fpr, tpr))

Area under curve :  0.4934789582768326


## AdaBoost

In [ ]:
#initialise SVM Linear
clf_pca_2 = AdaBoostClassifier(base_estimator= svm_model, learning_rate= 0.01, n_estimators= 3, algorithm='SAMME')


#train model
clf_pca_2.fit(pca_df.iloc[:len(X_resampled),:], y_resampled)

AdaBoostClassifier(algorithm='SAMME', base_estimator=LinearSVC(dual=False),
                   learning_rate=0.01, n_estimators=3)

In [ ]:
#get predictions and compare results
predictions_pca_2 = abc2.predict(X_test)

In [ ]:
cm_pca_2 = confusion_matrix(y_test, predictions_pca_2)
asr_pca_2 = accuracy_score(y_test, predictions_pca_2)
f1_pca_2 = f1_score(y_test, predictions_pca_2)
precision_pca_2 = precision_score(y_test, predictions_pca_2) 
recall_pca_2 = recall_score(y_test, predictions_pca_2)     
       
print('Confusion Matrix : ', '\n', cm_pca_2)
print('Accuracy score : ', asr_pca_2)
print('F1 score : ', f1_pca_2)
print('Precision score : ', precision_pca_2)
print('Recall score : ', recall_pca_2)


Confusion Matrix :  
 [[35662  1955]
 [ 1714  2135]]
Accuracy score :  0.9115178700622196
F1 score :  0.5378511147499686
Precision score :  0.5220048899755502
Recall score :  0.5546895297479865


In [ ]:
clf_pca_pred_2 =clf_pca_2.predict(pca_df.iloc[:len(X_test),:])
fpr, tpr, thresholds = roc_curve(y_test, clf_pca_pred_2, pos_label=1)
print('Aera under curve : ', auc(fpr, tpr))

Aera under curve :  0.49254565817004803


## Imbalanced Data Sampling Methods for SVM Linear kernel

There are 5 sections dedicated to handling imbalanced data as seen below.
1. None
2. SMOTE-NC
3. SMOTE
4. ENN
5. SMOTEENN

Read Data

In [ ]:
weather_full=pd.read_csv("weather_adjusted.csv")
weather_full

,timestamp,region,past_temperature,past_humidity,past_rainfall,past_wind_x,past_wind_y,delta_temperature,delta_humidity,delta_wind_x,delta_wind_y,rainfall
0,2017-01-01 03:00:00,central,26.850000,90.300000,0.0,-6.0,-4.0,-0.100000,-1.421085e-14,0.0,0.0,0.0
1,2017-01-01 03:00:00,east,26.125000,87.433333,0.0,-1.0,1.0,-0.150000,-5.333333e-01,0.0,1.0,0.0
2,2017-01-01 03:00:00,north,26.000000,87.000000,0.0,-1.0,2.0,-0.100000,4.000000e-01,0.0,0.0,0.0
3,2017-01-01 03:00:00,north-east,26.000000,89.250000,0.0,-1.0,1.0,0.033333,-1.500000e-01,-2.0,-1.0,0.0
4,2017-01-01 03:00:00,west,26.100000,87.066667,0.0,-4.0,0.0,-0.140000,8.666667e-01,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
197635,2021-12-31 23:00:00,central,23.666667,95.400000,1.0,-1.0,-1.0,-0.766667,-2.150000e+00,-3.0,1.0,1.0
197636,2021-12-31 23:00:00,east,24.800000,93.000000,1.0,-3.0,0.0,0.100000,-3.000000e-01,-2.0,-2.0,0.0
197637,2021-12-31 23:00:00,north,24.500000,96.200000,1.0,0.0,-1.0,0.200000,-6.000000e-01,-2.0,-1.0,0.0
197638,2021-12-31 23:00:00,north-east,24.300000,92.650000,0.0,0.0,-1.0,0.350000,-1.000000e+00,0.0,-3.0,0.0


Create time variables

In [ ]:
weather_full.timestamp=pd.to_datetime(weather_full.timestamp,infer_datetime_format=True)
weather_full["year"]=weather_full.timestamp.apply(lambda x: x.year)
weather_full["quarter"]=weather_full.timestamp.apply(lambda x: x.quarter)
weather_full["month"]=weather_full.timestamp.apply(lambda x: x.month)
weather_full["day"]=weather_full.timestamp.apply(lambda x: x.day)
weather_full["hour"]=weather_full.timestamp.apply(lambda x: x.hour)
weather_full

,timestamp,region,past_temperature,past_humidity,past_rainfall,past_wind_x,past_wind_y,delta_temperature,delta_humidity,delta_wind_x,delta_wind_y,rainfall,year,quarter,month,day,hour
0,2017-01-01 03:00:00,central,26.850000,90.300000,0.0,-6.0,-4.0,-0.100000,-1.421085e-14,0.0,0.0,0.0,2017,1,1,1,3
1,2017-01-01 03:00:00,east,26.125000,87.433333,0.0,-1.0,1.0,-0.150000,-5.333333e-01,0.0,1.0,0.0,2017,1,1,1,3
2,2017-01-01 03:00:00,north,26.000000,87.000000,0.0,-1.0,2.0,-0.100000,4.000000e-01,0.0,0.0,0.0,2017,1,1,1,3
3,2017-01-01 03:00:00,north-east,26.000000,89.250000,0.0,-1.0,1.0,0.033333,-1.500000e-01,-2.0,-1.0,0.0,2017,1,1,1,3
4,2017-01-01 03:00:00,west,26.100000,87.066667,0.0,-4.0,0.0,-0.140000,8.666667e-01,0.0,0.0,0.0,2017,1,1,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197635,2021-12-31 23:00:00,central,23.666667,95.400000,1.0,-1.0,-1.0,-0.766667,-2.150000e+00,-3.0,1.0,1.0,2021,4,12,31,23
197636,2021-12-31 23:00:00,east,24.800000,93.000000,1.0,-3.0,0.0,0.100000,-3.000000e-01,-2.0,-2.0,0.0,2021,4,12,31,23
197637,2021-12-31 23:00:00,north,24.500000,96.200000,1.0,0.0,-1.0,0.200000,-6.000000e-01,-2.0,-1.0,0.0,2021,4,12,31,23
197638,2021-12-31 23:00:00,north-east,24.300000,92.650000,0.0,0.0,-1.0,0.350000,-1.000000e+00,0.0,-3.0,0.0,2021,4,12,31,23


In [ ]:
weather_full=pd.concat([weather_full.iloc[:,12:],weather_full.iloc[:,:11],weather_full.iloc[:,11:12]],axis=1)
weather_full

,year,quarter,month,day,hour,timestamp,region,past_temperature,past_humidity,past_rainfall,past_wind_x,past_wind_y,delta_temperature,delta_humidity,delta_wind_x,delta_wind_y,rainfall
0,2017,1,1,1,3,2017-01-01 03:00:00,central,26.850000,90.300000,0.0,-6.0,-4.0,-0.100000,-1.421085e-14,0.0,0.0,0.0
1,2017,1,1,1,3,2017-01-01 03:00:00,east,26.125000,87.433333,0.0,-1.0,1.0,-0.150000,-5.333333e-01,0.0,1.0,0.0
2,2017,1,1,1,3,2017-01-01 03:00:00,north,26.000000,87.000000,0.0,-1.0,2.0,-0.100000,4.000000e-01,0.0,0.0,0.0
3,2017,1,1,1,3,2017-01-01 03:00:00,north-east,26.000000,89.250000,0.0,-1.0,1.0,0.033333,-1.500000e-01,-2.0,-1.0,0.0
4,2017,1,1,1,3,2017-01-01 03:00:00,west,26.100000,87.066667,0.0,-4.0,0.0,-0.140000,8.666667e-01,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197635,2021,4,12,31,23,2021-12-31 23:00:00,central,23.666667,95.400000,1.0,-1.0,-1.0,-0.766667,-2.150000e+00,-3.0,1.0,1.0
197636,2021,4,12,31,23,2021-12-31 23:00:00,east,24.800000,93.000000,1.0,-3.0,0.0,0.100000,-3.000000e-01,-2.0,-2.0,0.0
197637,2021,4,12,31,23,2021-12-31 23:00:00,north,24.500000,96.200000,1.0,0.0,-1.0,0.200000,-6.000000e-01,-2.0,-1.0,0.0
197638,2021,4,12,31,23,2021-12-31 23:00:00,north-east,24.300000,92.650000,0.0,0.0,-1.0,0.350000,-1.000000e+00,0.0,-3.0,0.0


### Train-test Splits

In [ ]:
runtimes=list(pd.date_range('2017-01-01 00:00:00',
                            '2021-12-31 23:59:59',
                            freq='60T'))
training_runtimes=runtimes[:int(0.8*len(runtimes))]
X_train = weather_full[weather_full["timestamp"].isin(training_runtimes)].iloc[:,:-1]
X_test = weather_full[~weather_full["timestamp"].isin(training_runtimes)].iloc[:,:-1]
y_train = weather_full[weather_full["timestamp"].isin(training_runtimes)][["rainfall"]]
y_test = weather_full[~weather_full["timestamp"].isin(training_runtimes)][["rainfall"]]
X_train=X_train.drop(columns=["timestamp"])
X_test=X_test.drop(columns=["timestamp"])

## Section 1: No Sampling

Dummifying and Scaling

In [ ]:
temp_df=pd.concat([X_train,X_test],axis=0)
temp_df=pd.get_dummies(temp_df, columns=["region"], prefix=["region"])
scaler=StandardScaler()
temp_df.iloc[:,5:14]=scaler.fit_transform(temp_df.iloc[:,5:14])
X_train = temp_df.iloc[:len(X_train),:]
X_test = temp_df.iloc[len(X_train):,:]
del temp_df

#### Vanilla SVM model (linear)

Converting y into a 1d array

In [ ]:
y_train_reshaped = np.array(y_train).flatten()
y_train_reshaped.shape

(156174,)

In [ ]:
y_test_reshaped = np.array(y_test).flatten()
y_test_reshaped.shape

(41466,)

In [ ]:
#build the base svm model

svm_model = LinearSVC(dual= False)
svm_model.fit(X_train, y_train_reshaped)
svm_model_pred = svm_model.predict(X_test)


print('Confusion Matrix :', '\n', confusion_matrix(y_test_reshaped, svm_model_pred))

#Accuracy score
asr_svm = accuracy_score(y_test_reshaped,svm_model_pred)
print('Accuracy score for SVM : ', asr_svm)

# Precision
precision_svm = precision_score(y_test_reshaped, svm_model_pred) 
print('Precision for SVM : ', precision_svm)

#Recall
recall_svm = recall_score(y_test_reshaped, svm_model_pred) 
print('Recall for SVM : ', recall_svm)

#F score
f1_svm = f1_score(y_test_reshaped,svm_model_pred)
print('F-score for SVM : ', f1_svm)
#AUC  ROC
auc_svm = roc_auc_score(y_test_reshaped, svm_model_pred)
print("area under curve (auc): ", auc_svm)


Confusion Matrix : 
 [[36401  1216]
 [ 2279  1570]]
Accuracy score for SVM :  0.9157140790044856
Precision for SVM :  0.5635319454414932
Recall for SVM :  0.4078981553650299
F-score for SVM :  0.47324792765636775
area under curve (auc):  0.6877861726129985


## Section 2. SMOTE-NC

Redo the steps:
1. Train-test split
2. Resampling
3. Dummyfying and Normalization
4. Modelling

In [ ]:
runtimes=list(pd.date_range('2017-01-01 00:00:00',
                            '2021-12-31 23:59:59',
                            freq='60T'))
training_runtimes=runtimes[:int(0.8*len(runtimes))]
X_train = weather_full[weather_full["timestamp"].isin(training_runtimes)].iloc[:,:-1]
X_test = weather_full[~weather_full["timestamp"].isin(training_runtimes)].iloc[:,:-1]
y_train = weather_full[weather_full["timestamp"].isin(training_runtimes)][["rainfall"]]
y_test = weather_full[~weather_full["timestamp"].isin(training_runtimes)][["rainfall"]]
X_train=X_train.drop(columns=["timestamp"])
X_test=X_test.drop(columns=["timestamp"])

In [ ]:
print(Counter(y_train.rainfall))
smote_nc = SMOTENC(categorical_features=[5], random_state=0)
X_resampled, y_resampled = smote_nc.fit_resample(X_train, y_train.rainfall)
Counter(y_resampled)

Counter({0.0: 143549, 1.0: 12625})


Counter({0.0: 143549, 1.0: 143549})

In [ ]:
temp_df=pd.concat([X_resampled,X_test],axis=0)
temp_df=pd.get_dummies(temp_df, columns=["region"], prefix=["region"])
scaler=StandardScaler()
temp_df.iloc[:,5:14]=scaler.fit_transform(temp_df.iloc[:,5:14])
X_resampled = temp_df.iloc[:len(X_resampled),:]
X_test = temp_df.iloc[len(X_resampled):,:]
del temp_df

In [ ]:
#build the base svm model

svm_model_1=LinearSVC(dual= False)

svm_model_1.fit(X_resampled, y_resampled)
svm_model_1_pred = svm_model_1.predict(X_test)

print('Confusion Matrix :', '\n', confusion_matrix(y_test, svm_model_1_pred))

#Accuracy score
asr_svm_1 = accuracy_score(y_test, svm_model_1_pred)
print('Accuracy score for SVM : ', asr_svm_1)

# Precision
precision_svm_1 = precision_score(y_test, svm_model_1_pred) 
print('Precision for SVM : ', precision_svm_1)

#Recall
recall_svm_1 = recall_score(y_test, svm_model_1_pred) 
print('Recall for SVM : ', recall_svm_1)

#F score
f1_svm_1 = f1_score(y_test,svm_model_1_pred)
print('F-score for SVM : ', f1_svm_1)

#AUC  ROC
auc_svm_1 = roc_auc_score(y_test, svm_model_1_pred)
print("area under curve (auc): ", auc_svm_1)


Confusion Matrix : 
 [[35277  2340]
 [ 1666  2183]]
Accuracy score for SVM :  0.9033907297544976
Precision for SVM :  0.4826442626575282
Recall for SVM :  0.567160301376981
F-score for SVM :  0.5215002388915432
area under curve (auc):  0.7524771919198486


## Section 3: SMOTE

Redo the steps:
1. Train-test split
2. Dummifying first since normal SMOTE cannot handle categorical
3. Resampling
3. Normalization
4. Modelling

In [ ]:
runtimes=list(pd.date_range('2017-01-01 00:00:00',
                            '2021-12-31 23:59:59',
                            freq='60T'))
training_runtimes=runtimes[:int(0.8*len(runtimes))]
X_train = weather_full[weather_full["timestamp"].isin(training_runtimes)].iloc[:,:-1]
X_test = weather_full[~weather_full["timestamp"].isin(training_runtimes)].iloc[:,:-1]
y_train = weather_full[weather_full["timestamp"].isin(training_runtimes)][["rainfall"]]
y_test = weather_full[~weather_full["timestamp"].isin(training_runtimes)][["rainfall"]]
X_train=X_train.drop(columns=["timestamp"])
X_test=X_test.drop(columns=["timestamp"])

In [ ]:
temp_df=pd.concat([X_train,X_test],axis=0)
temp_df=pd.get_dummies(temp_df, columns=["region"], prefix=["region"])
X_train = temp_df.iloc[:len(X_train),:]
X_test = temp_df.iloc[len(X_train):,:]
del temp_df

In [ ]:
print(Counter(y_train.rainfall))
smote = SMOTE(random_state=0)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train.rainfall)
Counter(y_resampled)

Counter({0.0: 143549, 1.0: 12625})


Counter({0.0: 143549, 1.0: 143549})

In [ ]:
temp_df=pd.concat([X_resampled,X_test],axis=0)
scaler=StandardScaler()
temp_df.iloc[:,5:14]=scaler.fit_transform(temp_df.iloc[:,5:14])
X_resampled = temp_df.iloc[:len(X_resampled),:]
X_test = temp_df.iloc[len(X_resampled):,:]
del temp_df

In [ ]:
svm_model_2=LinearSVC(dual= False)
svm_model_2.fit(X_resampled, y_resampled)
svm_model_2_pred = svm_model_2.predict(X_test)

print('Confusion Matrix :', '\n', confusion_matrix(y_test, svm_model_2_pred))

#Accuracy score
asr_svm_2 = accuracy_score(y_test, svm_model_2_pred)
print('Accuracy score for SVM : ', asr_svm_2)

#Precision
precision_svm_2 = precision_score(y_test, svm_model_2_pred) 
print('Precision for SVM : ', precision_svm_2)

#Recall
recall_svm_2 = recall_score(y_test, svm_model_2_pred) 
print('Recall for SVM : ', recall_svm_2)


#F score
f1_svm_2 = f1_score(y_test,svm_model_2_pred)
print('F-score for SVM : ', f1_svm_2)

#AUC ROC 
auc_svm_2 = roc_auc_score(y_test, svm_model_2_pred)
print("area under curve (auc): ", auc_svm_2)

Confusion Matrix : 
 [[35863  1754]
 [ 1756  2093]]
Accuracy score for SVM :  0.9153523368542903
Precision for SVM :  0.5440603067325188
Recall for SVM :  0.5437776045726163
F-score for SVM :  0.5439189189189189
area under curve (auc):  0.7485748750725484


In [ ]:
cross_val_score(svm_model_2, X_resampled, y_resampled, scoring="f1",cv=10).mean()

0.8806772460789004

## Section 4: Edited Nearest Neighbours (UnderSampling)

Redo the steps:
1. Train-test split
2. Dummifying first since ENN cannot handle categorical
3. Resampling
3. Normalization
4. Modelling

In [ ]:
runtimes=list(pd.date_range('2017-01-01 00:00:00',
                            '2021-12-31 23:59:59',
                            freq='60T'))
training_runtimes=runtimes[:int(0.8*len(runtimes))]
X_train = weather_full[weather_full["timestamp"].isin(training_runtimes)].iloc[:,:-1]
X_test = weather_full[~weather_full["timestamp"].isin(training_runtimes)].iloc[:,:-1]
y_train = weather_full[weather_full["timestamp"].isin(training_runtimes)][["rainfall"]]
y_test = weather_full[~weather_full["timestamp"].isin(training_runtimes)][["rainfall"]]
X_train=X_train.drop(columns=["timestamp"])
X_test=X_test.drop(columns=["timestamp"])

In [ ]:
temp_df=pd.concat([X_train,X_test],axis=0)
temp_df=pd.get_dummies(temp_df, columns=["region"], prefix=["region"])
X_train = temp_df.iloc[:len(X_train),:]
X_test = temp_df.iloc[len(X_train):,:]
del temp_df

In [ ]:
print(Counter(y_train.rainfall))
enn = EditedNearestNeighbours()
X_resampled, y_resampled = enn.fit_resample(X_train, y_train.rainfall)
Counter(y_resampled)

Counter({0.0: 143549, 1.0: 12625})


Counter({0.0: 125276, 1.0: 12625})

In [ ]:
temp_df=pd.concat([X_resampled,X_test],axis=0)
scaler=StandardScaler()
temp_df.iloc[:,5:14]=scaler.fit_transform(temp_df.iloc[:,5:14])
X_resampled = temp_df.iloc[:len(X_resampled),:]
X_test = temp_df.iloc[len(X_resampled):,:]
del temp_df

In [ ]:
# build the SVM model

svm_model_3 = LinearSVC(dual= False)
svm_model_3.fit(X_resampled, y_resampled)
svm_model_3_pred = svm_model_3.predict(X_test)


print('Confusion Matrix :', '\n', confusion_matrix(y_test, svm_model_3_pred))

#Accuracy score
asr_svm_3 = accuracy_score(y_test,svm_model_3_pred)
print('Accuracy score for SVM : ', asr_svm_3)

#Precision
precision_svm_3 = precision_score(y_test, svm_model_3_pred) 
print('Precision for SVM : ', precision_svm_3)

#Recall
recall_svm_3 = recall_score(y_test, svm_model_3_pred) 
print('Recall for SVM : ', recall_svm_3)

#F score
f1_svm_3 = f1_score(y_test,svm_model_3_pred)
print('F-score for SVM : ', f1_svm_3)

#AUC ROC 
auc_svm_3 = roc_auc_score(y_test, svm_model_3_pred)
print("area under curve (auc): ", auc_svm_3)

Confusion Matrix : 
 [[35863  1754]
 [ 1756  2093]]
Accuracy score for SVM :  0.9153523368542903
Precision for SVM :  0.5440603067325188
Recall for SVM :  0.5437776045726163
F-score for SVM :  0.5439189189189189
area under curve (auc):  0.7485748750725484


In [ ]:
cross_val_score(svm_model_3, X_resampled, y_resampled, scoring="f1",cv=10).mean()

0.5968969368591526

## Section 5: Using SMOTEENN

Redo the steps:
1. Train-test split
2. Dummifying first since SMOTEENN cannot handle categorical
3. Resampling
3. Normalization
4. Modelling

In [ ]:
runtimes=list(pd.date_range('2017-01-01 00:00:00',
                            '2021-12-31 23:59:59',
                            freq='60T'))
training_runtimes=runtimes[:int(0.8*len(runtimes))]
X_train = weather_full[weather_full["timestamp"].isin(training_runtimes)].iloc[:,:-1]
X_test = weather_full[~weather_full["timestamp"].isin(training_runtimes)].iloc[:,:-1]
y_train = weather_full[weather_full["timestamp"].isin(training_runtimes)][["rainfall"]]
y_test = weather_full[~weather_full["timestamp"].isin(training_runtimes)][["rainfall"]]
X_train=X_train.drop(columns=["timestamp"])
X_test=X_test.drop(columns=["timestamp"])

In [ ]:
temp_df=pd.concat([X_train,X_test],axis=0)
temp_df=pd.get_dummies(temp_df, columns=["region"], prefix=["region"])
X_train = temp_df.iloc[:len(X_train),:]
X_test = temp_df.iloc[len(X_train):,:]
del temp_df

In [ ]:
print(Counter(y_train.rainfall))
smoteenn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smoteenn.fit_resample(X_train, y_train.rainfall)
Counter(y_resampled)

Counter({0.0: 143549, 1.0: 12625})


Counter({0.0: 109258, 1.0: 142925})

In [ ]:
temp_df=pd.concat([X_resampled,X_test],axis=0)
scaler=StandardScaler()
temp_df.iloc[:,5:14]=scaler.fit_transform(temp_df.iloc[:,5:14])
X_resampled = temp_df.iloc[:len(X_resampled),:]
X_test = temp_df.iloc[len(X_resampled):,:]
del temp_df

In [ ]:
# build the SVM model

svm_model_4 = LinearSVC(dual= False)
svm_model_4.fit(X_resampled, y_resampled)
svm_model_4_pred = svm_model_4.predict(X_test)

print('Confusion Matrix :', '\n', confusion_matrix(y_test, svm_model_4_pred))

# Accuracy score
asr_svm_4 = accuracy_score(y_test,svm_model_4_pred)
print('Accuracy score for SVM : ', asr_svm_4)

# Precision
precision_svm_4 = precision_score(y_test, svm_model_4_pred) 
print('Precision for SVM : ', precision_svm_4)

# Recall
recall_svm_4 = recall_score(y_test, svm_model_4_pred) 
print('Recall for SVM : ', recall_svm_4)

# F score
f1_svm_4 = f1_score(y_test,svm_model_4_pred)
print('F-score for SVM : ', f1_svm_4)

# AUC ROC 
auc_svm_4 = roc_auc_score(y_test, svm_model_4_pred)
print("area under curve (auc): ", auc_svm_4)

Confusion Matrix : 
 [[35862  1755]
 [ 1756  2093]]
Accuracy score for SVM :  0.9153282207109439
Precision for SVM :  0.543918918918919
Recall for SVM :  0.5437776045726163
F-score for SVM :  0.5438482525659348
area under curve (auc):  0.7485615832098268


In [ ]:
cross_val_score(svm_model_4, X_resampled, y_resampled, scoring="f1",cv=10).mean()

0.9018983888536013

## Imbalanced Data Sampling Methods for AdaBoost on the SVM Linear kernel

There are 5 sections dedicated to handling imblanaced data as seen below.
1. None
2. SMOTE-NC
3. SMOTE
4. ENN
5. SMOTEENN

Read Data

In [ ]:
weather_full=pd.read_csv("weather_adjusted.csv")
weather_full

,timestamp,region,past_temperature,past_humidity,past_rainfall,past_wind_x,past_wind_y,delta_temperature,delta_humidity,delta_wind_x,delta_wind_y,rainfall
0,2017-01-01 03:00:00,central,26.850000,90.300000,0.0,-6.0,-4.0,-0.100000,-1.421085e-14,0.0,0.0,0.0
1,2017-01-01 03:00:00,east,26.125000,87.433333,0.0,-1.0,1.0,-0.150000,-5.333333e-01,0.0,1.0,0.0
2,2017-01-01 03:00:00,north,26.000000,87.000000,0.0,-1.0,2.0,-0.100000,4.000000e-01,0.0,0.0,0.0
3,2017-01-01 03:00:00,north-east,26.000000,89.250000,0.0,-1.0,1.0,0.033333,-1.500000e-01,-2.0,-1.0,0.0
4,2017-01-01 03:00:00,west,26.100000,87.066667,0.0,-4.0,0.0,-0.140000,8.666667e-01,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
197635,2021-12-31 23:00:00,central,23.666667,95.400000,1.0,-1.0,-1.0,-0.766667,-2.150000e+00,-3.0,1.0,1.0
197636,2021-12-31 23:00:00,east,24.800000,93.000000,1.0,-3.0,0.0,0.100000,-3.000000e-01,-2.0,-2.0,0.0
197637,2021-12-31 23:00:00,north,24.500000,96.200000,1.0,0.0,-1.0,0.200000,-6.000000e-01,-2.0,-1.0,0.0
197638,2021-12-31 23:00:00,north-east,24.300000,92.650000,0.0,0.0,-1.0,0.350000,-1.000000e+00,0.0,-3.0,0.0


Create time variables

In [ ]:
weather_full.timestamp=pd.to_datetime(weather_full.timestamp,infer_datetime_format=True)
weather_full["year"]=weather_full.timestamp.apply(lambda x: x.year)
weather_full["quarter"]=weather_full.timestamp.apply(lambda x: x.quarter)
weather_full["month"]=weather_full.timestamp.apply(lambda x: x.month)
weather_full["day"]=weather_full.timestamp.apply(lambda x: x.day)
weather_full["hour"]=weather_full.timestamp.apply(lambda x: x.hour)
weather_full

,timestamp,region,past_temperature,past_humidity,past_rainfall,past_wind_x,past_wind_y,delta_temperature,delta_humidity,delta_wind_x,delta_wind_y,rainfall,year,quarter,month,day,hour
0,2017-01-01 03:00:00,central,26.850000,90.300000,0.0,-6.0,-4.0,-0.100000,-1.421085e-14,0.0,0.0,0.0,2017,1,1,1,3
1,2017-01-01 03:00:00,east,26.125000,87.433333,0.0,-1.0,1.0,-0.150000,-5.333333e-01,0.0,1.0,0.0,2017,1,1,1,3
2,2017-01-01 03:00:00,north,26.000000,87.000000,0.0,-1.0,2.0,-0.100000,4.000000e-01,0.0,0.0,0.0,2017,1,1,1,3
3,2017-01-01 03:00:00,north-east,26.000000,89.250000,0.0,-1.0,1.0,0.033333,-1.500000e-01,-2.0,-1.0,0.0,2017,1,1,1,3
4,2017-01-01 03:00:00,west,26.100000,87.066667,0.0,-4.0,0.0,-0.140000,8.666667e-01,0.0,0.0,0.0,2017,1,1,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197635,2021-12-31 23:00:00,central,23.666667,95.400000,1.0,-1.0,-1.0,-0.766667,-2.150000e+00,-3.0,1.0,1.0,2021,4,12,31,23
197636,2021-12-31 23:00:00,east,24.800000,93.000000,1.0,-3.0,0.0,0.100000,-3.000000e-01,-2.0,-2.0,0.0,2021,4,12,31,23
197637,2021-12-31 23:00:00,north,24.500000,96.200000,1.0,0.0,-1.0,0.200000,-6.000000e-01,-2.0,-1.0,0.0,2021,4,12,31,23
197638,2021-12-31 23:00:00,north-east,24.300000,92.650000,0.0,0.0,-1.0,0.350000,-1.000000e+00,0.0,-3.0,0.0,2021,4,12,31,23


In [ ]:
weather_full=pd.concat([weather_full.iloc[:,12:],weather_full.iloc[:,:11],weather_full.iloc[:,11:12]],axis=1)
weather_full

,year,quarter,month,day,hour,timestamp,region,past_temperature,past_humidity,past_rainfall,past_wind_x,past_wind_y,delta_temperature,delta_humidity,delta_wind_x,delta_wind_y,rainfall
0,2017,1,1,1,3,2017-01-01 03:00:00,central,26.850000,90.300000,0.0,-6.0,-4.0,-0.100000,-1.421085e-14,0.0,0.0,0.0
1,2017,1,1,1,3,2017-01-01 03:00:00,east,26.125000,87.433333,0.0,-1.0,1.0,-0.150000,-5.333333e-01,0.0,1.0,0.0
2,2017,1,1,1,3,2017-01-01 03:00:00,north,26.000000,87.000000,0.0,-1.0,2.0,-0.100000,4.000000e-01,0.0,0.0,0.0
3,2017,1,1,1,3,2017-01-01 03:00:00,north-east,26.000000,89.250000,0.0,-1.0,1.0,0.033333,-1.500000e-01,-2.0,-1.0,0.0
4,2017,1,1,1,3,2017-01-01 03:00:00,west,26.100000,87.066667,0.0,-4.0,0.0,-0.140000,8.666667e-01,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197635,2021,4,12,31,23,2021-12-31 23:00:00,central,23.666667,95.400000,1.0,-1.0,-1.0,-0.766667,-2.150000e+00,-3.0,1.0,1.0
197636,2021,4,12,31,23,2021-12-31 23:00:00,east,24.800000,93.000000,1.0,-3.0,0.0,0.100000,-3.000000e-01,-2.0,-2.0,0.0
197637,2021,4,12,31,23,2021-12-31 23:00:00,north,24.500000,96.200000,1.0,0.0,-1.0,0.200000,-6.000000e-01,-2.0,-1.0,0.0
197638,2021,4,12,31,23,2021-12-31 23:00:00,north-east,24.300000,92.650000,0.0,0.0,-1.0,0.350000,-1.000000e+00,0.0,-3.0,0.0


### Train-test Splits

In [ ]:
runtimes=list(pd.date_range('2017-01-01 00:00:00',
                            '2021-12-31 23:59:59',
                            freq='60T'))
training_runtimes=runtimes[:int(0.8*len(runtimes))]
X_train = weather_full[weather_full["timestamp"].isin(training_runtimes)].iloc[:,:-1]
X_test = weather_full[~weather_full["timestamp"].isin(training_runtimes)].iloc[:,:-1]
y_train = weather_full[weather_full["timestamp"].isin(training_runtimes)][["rainfall"]]
y_test = weather_full[~weather_full["timestamp"].isin(training_runtimes)][["rainfall"]]
X_train=X_train.drop(columns=["timestamp"])
X_test=X_test.drop(columns=["timestamp"])

## Section 1: No Sampling

Dummifying and Scaling

In [ ]:
temp_df=pd.concat([X_train,X_test],axis=0)
temp_df=pd.get_dummies(temp_df, columns=["region"], prefix=["region"])
scaler=StandardScaler()
temp_df.iloc[:,5:14]=scaler.fit_transform(temp_df.iloc[:,5:14])
X_train = temp_df.iloc[:len(X_train),:]
X_test = temp_df.iloc[len(X_train):,:]
del temp_df

#### Creating the vanilla SVM model (linear)

converting y into a 1d array

In [ ]:
y_train_reshaped = np.array(y_train).flatten()
y_train_reshaped.shape

(156174,)

In [ ]:
y_test_reshaped = np.array(y_test).flatten()
y_test_reshaped.shape

(41466,)

Build the base svm model

In [ ]:
svm_model = LinearSVC(dual= False)
svm_model.fit(X_train, y_train_reshaped)
svm_model_pred = svm_model.predict(X_test)

Applying Adaboost on the svm model

In [ ]:
#AdaBoost the SVM Model
abc = AdaBoostClassifier(base_estimator= svm_model, learning_rate= 0.01, n_estimators= 2, algorithm='SAMME')
abc.fit(X_test, y_test_reshaped)
abc_pred = abc.predict(X_test)

cm_abc = confusion_matrix(y_test_reshaped, abc_pred)
asr_abc = accuracy_score(y_test_reshaped, abc_pred)
f1_abc = f1_score(y_test_reshaped, abc_pred)
precision_abc = precision_score(y_test_reshaped, abc_pred) 
recall_abc = recall_score(y_test_reshaped, abc_pred)     
       
print('Confusion Matrix : ', '\n', cm_abc)
print('Accuracy score : ', asr_abc)
print('F1 score : ', f1_abc)
print('Precision score : ', precision_abc)
print('Recall score : ', recall_abc)

y_preds= abc.predict_proba(X_test)
fpr, tpr , threshold = roc_curve(y_test_reshaped, y_preds[:,1,None], pos_label=1)
print('AUC ROC : ', auc(fpr,tpr))

Confusion Matrix :  
 [[37604    13]
 [ 3818    31]]
Accuracy score :  0.90761105484011
F1 score :  0.015926021063447215
Precision score :  0.7045454545454546
Recall score :  0.00805404001039231
AUC ROC :  0.5063195572517478


In [ ]:
cross_val_score(abc, X_train, y_train_reshaped, scoring="f1",cv=10).mean()

0.0050388028329680125

## Section 2. SMOTE-NC

Redo the steps:
1. Train-test split
2. Resampling
3. Dummyfying and Normalization
4. Modelling

In [ ]:
runtimes=list(pd.date_range('2017-01-01 00:00:00',
                            '2021-12-31 23:59:59',
                            freq='60T'))
training_runtimes=runtimes[:int(0.8*len(runtimes))]
X_train = weather_full[weather_full["timestamp"].isin(training_runtimes)].iloc[:,:-1]
X_test = weather_full[~weather_full["timestamp"].isin(training_runtimes)].iloc[:,:-1]
y_train = weather_full[weather_full["timestamp"].isin(training_runtimes)][["rainfall"]]
y_test = weather_full[~weather_full["timestamp"].isin(training_runtimes)][["rainfall"]]
X_train=X_train.drop(columns=["timestamp"])
X_test=X_test.drop(columns=["timestamp"])

In [ ]:
print(Counter(y_train.rainfall))
smote_nc = SMOTENC(categorical_features=[5], random_state=0)
X_resampled, y_resampled = smote_nc.fit_resample(X_train, y_train.rainfall)
Counter(y_resampled)

Counter({0.0: 143549, 1.0: 12625})


Counter({0.0: 143549, 1.0: 143549})

In [ ]:
temp_df=pd.concat([X_resampled,X_test],axis=0)
temp_df=pd.get_dummies(temp_df, columns=["region"], prefix=["region"])
scaler=StandardScaler()
temp_df.iloc[:,5:14]=scaler.fit_transform(temp_df.iloc[:,5:14])
X_resampled = temp_df.iloc[:len(X_resampled),:]
X_test = temp_df.iloc[len(X_resampled):,:]
del temp_df

In [ ]:
#AdaBoost the SVM Model

abc_1 = AdaBoostClassifier(base_estimator= svm_model, learning_rate= 0.01, n_estimators= 2, algorithm='SAMME')
abc_1.fit(X_resampled, y_resampled)
abc_1_pred = abc_1.predict(X_test)

cm_abc_1 = confusion_matrix(y_test, abc_1_pred)
asr_abc_1 = accuracy_score(y_test, abc_1_pred)
f1_abc_1 = f1_score(y_test, abc_1_pred)
precision_abc_1 = precision_score(y_test, abc_1_pred) 
recall_abc_1 = recall_score(y_test, abc_1_pred)     
       
print('Confusion Matrix : ', '\n', cm_abc_1)
print('Accuracy score : ', asr_abc_1)
print('F1 score : ', f1_abc_1)
print('Precision score : ', precision_abc_1)
print('Recall score : ', recall_abc_1)

y_preds= abc_1.predict_proba(X_test)
fpr, tpr , threshold = roc_curve(y_test, y_preds[:,1,None], pos_label=1)
print('AUC ROC : ' , auc(fpr,tpr))

Confusion Matrix :  
 [[35683  1934]
 [ 1715  2134]]
Accuracy score :  0.9120001929291468
F1 score :  0.5390930908172287
Precision score :  0.5245821042281219
Recall score :  0.5544297220057157
AUC ROC :  0.7515072519940263


In [ ]:
cross_val_score(abc_1, X_resampled, y_resampled, scoring="f1",cv=10).mean()

0.7211051489086706

## Section 3: SMOTE

Redo the steps:
1. Train-test split
2. Dummifying first since normal SMOTE cannot handle categorical
3. Resampling
3. Normalization
4. Modelling

In [ ]:
runtimes=list(pd.date_range('2017-01-01 00:00:00',
                            '2021-12-31 23:59:59',
                            freq='60T'))
training_runtimes=runtimes[:int(0.8*len(runtimes))]
X_train = weather_full[weather_full["timestamp"].isin(training_runtimes)].iloc[:,:-1]
X_test = weather_full[~weather_full["timestamp"].isin(training_runtimes)].iloc[:,:-1]
y_train = weather_full[weather_full["timestamp"].isin(training_runtimes)][["rainfall"]]
y_test = weather_full[~weather_full["timestamp"].isin(training_runtimes)][["rainfall"]]
X_train=X_train.drop(columns=["timestamp"])
X_test=X_test.drop(columns=["timestamp"])

In [ ]:
temp_df=pd.concat([X_train,X_test],axis=0)
temp_df=pd.get_dummies(temp_df, columns=["region"], prefix=["region"])
X_train = temp_df.iloc[:len(X_train),:]
X_test = temp_df.iloc[len(X_train):,:]
del temp_df

In [ ]:
print(Counter(y_train.rainfall))
smote = SMOTE(random_state=0)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train.rainfall)
Counter(y_resampled)

Counter({0.0: 143549, 1.0: 12625})


Counter({0.0: 143549, 1.0: 143549})

In [ ]:
temp_df=pd.concat([X_resampled,X_test],axis=0)
scaler=StandardScaler()
temp_df.iloc[:,5:14]=scaler.fit_transform(temp_df.iloc[:,5:14])
X_resampled = temp_df.iloc[:len(X_resampled),:]
X_test = temp_df.iloc[len(X_resampled):,:]
del temp_df

In [ ]:
#AdaBoost the SVM Model
abc_2 = AdaBoostClassifier(base_estimator= svm_model, learning_rate= 0.01, n_estimators= 2, algorithm='SAMME')
abc_2.fit(X_resampled, y_resampled)
abc_2_pred = abc_2.predict(X_test)

cm_abc_2 = confusion_matrix(y_test, abc_2_pred)
asr_abc_2 = accuracy_score(y_test, abc_2_pred)
f1_abc_2 = f1_score(y_test, abc_2_pred)
precision_abc_2 = precision_score(y_test, abc_2_pred) 
recall_abc_2 = recall_score(y_test, abc_2_pred)     
       
print('Confusion Matrix : ', '\n', cm_abc_2)
print('Accuracy score : ', asr_abc_2)
print('F1 score : ', f1_abc_2)
print('Precision score : ', precision_abc_2)
print('Recall score : ', recall_abc_2)

y_preds= abc_2.predict_proba(X_test)
fpr, tpr , threshold = roc_curve(y_test, y_preds[:,1,None], pos_label=1)
print('AUC ROC : ' , auc(fpr,tpr))

Confusion Matrix :  
 [[35809  1808]
 [ 1731  2118]]
Accuracy score :  0.9146529686972459
F1 score :  0.5448231511254019
Precision score :  0.5394803871625063
Recall score :  0.5502727981293842
AUC ROC :  0.7510628672783576


In [ ]:
cross_val_score(abc_2, X_resampled, y_resampled, scoring="f1",cv=10).mean()

0.7344195639225742

## Section 4: Edited Nearest Neighbours (UnderSampling)

Redo the steps:
1. Train-test split
2. Dummifying first since ENN cannot handle categorical
3. Resampling
3. Normalization
4. Modelling

In [ ]:
runtimes=list(pd.date_range('2017-01-01 00:00:00',
                            '2021-12-31 23:59:59',
                            freq='60T'))
training_runtimes=runtimes[:int(0.8*len(runtimes))]
X_train = weather_full[weather_full["timestamp"].isin(training_runtimes)].iloc[:,:-1]
X_test = weather_full[~weather_full["timestamp"].isin(training_runtimes)].iloc[:,:-1]
y_train = weather_full[weather_full["timestamp"].isin(training_runtimes)][["rainfall"]]
y_test = weather_full[~weather_full["timestamp"].isin(training_runtimes)][["rainfall"]]
X_train=X_train.drop(columns=["timestamp"])
X_test=X_test.drop(columns=["timestamp"])

In [ ]:
temp_df=pd.concat([X_train,X_test],axis=0)
temp_df=pd.get_dummies(temp_df, columns=["region"], prefix=["region"])
X_train = temp_df.iloc[:len(X_train),:]
X_test = temp_df.iloc[len(X_train):,:]
del temp_df

In [ ]:
print(Counter(y_train.rainfall))
enn = EditedNearestNeighbours()
X_resampled, y_resampled = enn.fit_resample(X_train, y_train.rainfall)
Counter(y_resampled)

Counter({0.0: 143549, 1.0: 12625})


Counter({0.0: 125276, 1.0: 12625})

In [ ]:
temp_df=pd.concat([X_resampled,X_test],axis=0)
scaler=StandardScaler()
temp_df.iloc[:,5:14]=scaler.fit_transform(temp_df.iloc[:,5:14])
X_resampled = temp_df.iloc[:len(X_resampled),:]
X_test = temp_df.iloc[len(X_resampled):,:]
del temp_df

In [ ]:
#AdaBoost the SVM Model
abc_3 = AdaBoostClassifier(base_estimator= svm_model, learning_rate= 0.01, n_estimators= 2, algorithm='SAMME')
abc_3.fit(X_resampled, y_resampled)
abc_3_pred = abc_3.predict(X_test)

cm_abc_3 = confusion_matrix(y_test, abc_3_pred)
asr_abc_3 = accuracy_score(y_test, abc_3_pred)
f1_abc_3 = f1_score(y_test, abc_3_pred)
precision_abc_3 = precision_score(y_test, abc_3_pred) 
recall_abc_3 = recall_score(y_test, abc_3_pred)     
       
print('Confusion Matrix : ', '\n', cm_abc_3)
print('Accuracy score : ', asr_abc_3)
print('F1 score : ', f1_abc_3)
print('Precision score : ', precision_abc_3)
print('Recall score : ', recall_abc_3)

y_preds= abc_3.predict_proba(X_test)
fpr, tpr , threshold = roc_curve(y_test, y_preds[:,1,None], pos_label=1)
print('AUC ROC : ' , auc(fpr,tpr))

Confusion Matrix :  
 [[36854   763]
 [ 2721  1128]]
Accuracy score :  0.9159793565812955
F1 score :  0.39303135888501745
Precision score :  0.5965097831835008
Recall score :  0.2930631332813718
AUC ROC :  0.6425986981931003


In [ ]:
cross_val_score(abc_3, X_resampled, y_resampled, scoring="f1",cv=10).mean()

0.41253039394700775

## Section 5: Using SMOTEENN

Redo the steps:
1. Train-test split
2. Dummifying first since SMOTEENN cannot handle categorical
3. Resampling
3. Normalization
4. Modelling

In [ ]:
runtimes=list(pd.date_range('2017-01-01 00:00:00',
                            '2021-12-31 23:59:59',
                            freq='60T'))
training_runtimes=runtimes[:int(0.8*len(runtimes))]
X_train = weather_full[weather_full["timestamp"].isin(training_runtimes)].iloc[:,:-1]
X_test = weather_full[~weather_full["timestamp"].isin(training_runtimes)].iloc[:,:-1]
y_train = weather_full[weather_full["timestamp"].isin(training_runtimes)][["rainfall"]]
y_test = weather_full[~weather_full["timestamp"].isin(training_runtimes)][["rainfall"]]
X_train=X_train.drop(columns=["timestamp"])
X_test=X_test.drop(columns=["timestamp"])

In [ ]:
temp_df=pd.concat([X_train,X_test],axis=0)
temp_df=pd.get_dummies(temp_df, columns=["region"], prefix=["region"])
X_train = temp_df.iloc[:len(X_train),:]
X_test = temp_df.iloc[len(X_train):,:]
del temp_df

In [ ]:
print(Counter(y_train.rainfall))
smoteenn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smoteenn.fit_resample(X_train, y_train.rainfall)
Counter(y_resampled)

Counter({0.0: 143549, 1.0: 12625})


Counter({0.0: 109258, 1.0: 142925})

In [ ]:
temp_df=pd.concat([X_resampled,X_test],axis=0)
scaler=StandardScaler()
temp_df.iloc[:,5:14]=scaler.fit_transform(temp_df.iloc[:,5:14])
X_resampled = temp_df.iloc[:len(X_resampled),:]
X_test = temp_df.iloc[len(X_resampled):,:]
del temp_df

In [ ]:
#AdaBoost the SVM Model
abc_4 = AdaBoostClassifier(base_estimator= svm_model, learning_rate= 0.01, n_estimators= 2, algorithm='SAMME')
abc_4.fit(X_resampled, y_resampled)
abc_4_pred = abc_4.predict(X_test)

cm_abc_4 = confusion_matrix(y_test, abc_4_pred)
asr_abc_4 = accuracy_score(y_test, abc_4_pred)
f1_abc_4 = f1_score(y_test, abc_4_pred)
precision_abc_4 = precision_score(y_test, abc_4_pred) 
recall_abc_4 = recall_score(y_test, abc_4_pred)     
       
print('Confusion Matrix : ', '\n', cm_abc_4)
print('Accuracy score : ', asr_abc_4)
print('F1 score : ', f1_abc_4)
print('Precision score : ', precision_abc_4)
print('Recall score : ', recall_abc_4)

y_preds= abc_4.predict_proba(X_test)
fpr, tpr , threshold = roc_curve(y_test, y_preds[:,1,None], pos_label=1)
print('AUC ROC: ', auc(fpr,tpr))

Confusion Matrix :  
 [[32741  4876]
 [ 1503  2346]]
Accuracy score :  0.8461631215935947
F1 score :  0.42380995393370063
Precision score :  0.3248407643312102
Recall score :  0.6095089633671084
AUC ROC:  0.7407413853621251


In [ ]:
cross_val_score(abc_4, X_resampled, y_resampled, scoring="f1",cv=10).mean()

0.7803208601210336

## <center>Summary for SVM linear kernel</center>
|         | None | SMOTE-NC | ENN | SMOTEENN |
|:-------:|:-------:|:------------------:|:--------:|:----------------------------:|
| F1   |  0.473 |        0.522       |   0.544  |             0.544            |
| Recall  |  0.408  |        0.567       |   0.544  |             0.544            |
| ROC AUC |  0.688  |        0.752       |   0.749  |             0.749            |
|CV Score |0.443  |        0.430       |   0.597  |             0.902            |

## <center>Summary for Adaboosted SVM linear kernel</center>
|         | None | SMOTE-NC | ENN | SMOTEENN |
|:-------:|:-------:|:------------------:|:--------:|:----------------------------:|
| F1   |  0.016 |        0.539       |   0.393  |             0.424            |
| Recall  |  0.008  |        0.554       |   0.293  |             0.610            |
| ROC AUC |  0.506  |        0.752       |   0.643  |             0.741            |
|CV Score |0.005  |        0.721        |   0.413  |             0.780            |